In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [26]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

base_url = 'http://operadata.stanford.edu/?f%5Bgenre_facet%5D%5B%5D='
genres = ['Opera', 'Oratorio', 'Operetta']

In [29]:
works = []

for genre in genres:
    print(genre)
    url = base_url + genre + '&per_page=100'
    r  = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html5lib")
    
    try:
        pages = soup.find('div', attrs={'class': 'pagination'}).find_all('li')[-1]
        pages = pages.get_text().strip().replace(',', '')
        pages = int(pages)
    except:
        pages = 1
    
    for page in range(1, pages+1):
        if page%10==0:
            print(page)
            
#         print(url)
        p_url = url + '&page=' + str(page)
        r  = requests.get(p_url, headers=headers)
        soup = BeautifulSoup(r.text, "html5lib")
        results = soup.find('div', attrs={'id': 'documents'})
        results = results.find_all('div', attrs={'class': 'document'})

        for result in results:
            title = result.find('h5', attrs={'class': 'index_title'}).find('a')            
            name = title.get_text().strip()
            w_url = title.get('href')
            
            composer, country, year = '', '', 2099
            data = result.find('dl', attrs={'class': 'document-metadata'})
            data = list(zip(data.find_all('dt'),data.find_all('dd')))            
            for item in data:
                attribute, value = item
                attribute = attribute.get_text().strip()
                value     = value.get_text().strip()                  
                
                if attribute=='Composer:':
                    composer = value
                elif attribute=='Year:':
                    year = value                    
                elif attribute=='Country:':
                    country = value                        
                
            opera = []
            opera.append(genre)
            opera.append(name)
            opera.append(w_url)
            opera.append(composer)
            opera.append(year)
            opera.append(country)                
#             print(opera)
            works.append(opera)            

#Saving results into dataframe and folder            
import pandas as pd                     
headers = ['genre', 'name', 'uri', 'composer', 'year', 'country']
df = pd.DataFrame(works, columns=headers)
df.to_csv('../../data/processed/works/stanford.csv')

Opera
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
Oratorio
10
20
30
40
50
Operetta


FileNotFoundError: [Errno 2] No such file or directory: '../../data/processed/workds/stanford.csv'

In [35]:
hist = df.hist(column=['year'])

ValueError: num must be 1 <= num <= 0, not 1

In [38]:
# df = df.groupby(['year']).agg({'composer': 'count'})

HTML(df.to_html(escape=False))


,composer
year,
2099,821
1589,1
1590,2
1595,1
1600,4
1602,1
1606,1
1607,1
1608,4
